In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_18.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,NYM,6/18/2021,1,542340,Jonathan Villar,1,N,NaN
1,NYM,6/18/2021,1,596019,Francisco Lindor,2,N,NaN
2,NYM,6/18/2021,1,624413,Peter Alonso,3,N,NaN
3,NYM,6/18/2021,1,642086,Dominic Smith,4,N,NaN
4,NYM,6/18/2021,1,543510,James McCann,5,N,NaN
...,...,...,...,...,...,...,...,...
278,SEA,6/18/2021,1,664238,Dylan Moore,6,N,NaN
279,SEA,6/18/2021,1,641343,Jake Bauers,7,N,NaN
280,SEA,6/18/2021,1,608596,Tom Murphy,8,N,NaN
281,SEA,6/18/2021,1,666211,Taylor Trammell,9,N,NaN


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'WAS': 'NYM',
 'NYM': 'WAS',
 'BAL': 'TOR',
 'TOR': 'BAL',
 'PIT': 'CLE',
 'CLE': 'PIT',
 'NYY': 'OAK',
 'OAK': 'NYY',
 'ATL': 'STL',
 'STL': 'ATL',
 'TEX': 'MIN',
 'MIN': 'TEX',
 'CHC': 'MIA',
 'MIA': 'CHC',
 'HOU': 'CHW',
 'CHW': 'HOU',
 'KC': 'BOS',
 'BOS': 'KC',
 'COL': 'MIL',
 'MIL': 'COL',
 'LAA': 'DET',
 'DET': 'LAA',
 'ARI': 'LAD',
 'LAD': 'ARI',
 'SF': 'PHI',
 'PHI': 'SF',
 'SD': 'CIN',
 'CIN': 'SD',
 'SEA': 'TB',
 'TB': 'SEA'}

In [12]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    if len(lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values) > 0: 
        opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                                & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
        if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
                opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
                opp_pitcher.append(opp_pitch)
                keep_pitch.append(opp_pitch)
        else:
            del_indexes.append(i)
    else:
        del_indexes.append(i)
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-12-a4b56b56add5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-12-a4b56b56add5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [13]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-13-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [14]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,NYM,6/18/2021,1,542340,Jonathan Villar,1,N,NaN,R,Erick Fedde,...,0.233010,0.333333,0.330097,0.663430,0.097087,0.312057,25.331105,-4.045609,0.295915,88.139325
1,NYM,6/18/2021,1,596019,Francisco Lindor,2,N,NaN,R,Erick Fedde,...,0.219178,0.306306,0.349315,0.655621,0.130137,0.241228,33.160730,-7.451873,0.287484,81.316404
2,NYM,6/18/2021,1,624413,Peter Alonso,3,N,NaN,R,Erick Fedde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYM,6/18/2021,1,642086,Dominic Smith,4,N,NaN,R,Erick Fedde,...,0.283898,0.332031,0.504237,0.836269,0.220339,0.335294,39.209169,8.155034,0.352270,126.626800
4,NYM,6/18/2021,1,543510,James McCann,5,N,NaN,R,Erick Fedde,...,0.208589,0.269663,0.319018,0.588681,0.110429,0.276190,13.833052,-7.590696,0.260546,66.900327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,SEA,6/18/2021,1,641584,Jake Fraley,5,N,NaN,R,Michael Wacha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,SEA,6/18/2021,1,664238,Dylan Moore,6,N,NaN,R,Michael Wacha,...,0.210227,0.317073,0.420455,0.737528,0.210227,0.269231,26.515307,1.648680,0.322889,109.697295
279,SEA,6/18/2021,1,641343,Jake Bauers,7,N,NaN,R,Michael Wacha,...,0.184211,0.295455,0.289474,0.584928,0.105263,0.222222,7.167731,-3.092486,0.265402,65.885243
280,SEA,6/18/2021,1,608596,Tom Murphy,8,N,NaN,R,Michael Wacha,...,0.132075,0.178571,0.264151,0.442722,0.132075,0.156250,1.470206,-5.059023,0.196329,25.813338


In [15]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [16]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [17]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [18]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,NYM,6/18/2021,1,542340,Jonathan Villar,1,N,NaN,R,Erick Fedde,...,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857
1,NYM,6/18/2021,1,596019,Francisco Lindor,2,N,NaN,R,Erick Fedde,...,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857
2,NYM,6/18/2021,1,624413,Peter Alonso,3,N,NaN,R,Erick Fedde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYM,6/18/2021,1,642086,Dominic Smith,4,N,NaN,R,Erick Fedde,...,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857,5.142857
4,NYM,6/18/2021,1,543510,James McCann,5,N,NaN,R,Erick Fedde,...,7.552448,7.552448,7.552448,7.552448,7.552448,7.552448,7.552448,7.552448,7.552448,7.552448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,SEA,6/18/2021,1,641584,Jake Fraley,5,N,NaN,R,Michael Wacha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,SEA,6/18/2021,1,664238,Dylan Moore,6,N,NaN,R,Michael Wacha,...,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976
279,SEA,6/18/2021,1,641343,Jake Bauers,7,N,NaN,R,Michael Wacha,...,8.678571,8.678571,8.678571,8.678571,8.678571,8.678571,8.678571,8.678571,8.678571,8.678571
280,SEA,6/18/2021,1,608596,Tom Murphy,8,N,NaN,R,Michael Wacha,...,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976,8.560976


In [19]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [20]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [21]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
8,NYM,6/18/2021,1,664192,Joey Lucchesi,SP,N,NaN,2,6,...,6.311688,1.380531,0.530973,0.849558,1.714286,8.883117,2.054794,3.275862,23.546949,29.333280
17,WAS,6/18/2021,1,607200,Erick Fedde,SP,N,NaN,4,4,...,7.552448,0.751174,0.356808,0.394366,0.972973,5.370629,1.000000,2.740741,22.659825,18.918107
27,TOR,6/18/2021,1,592662,Robbie Ray,SP,N,NaN,1,8,...,19.555556,2.129032,1.118280,1.010753,2.107936,13.333333,2.477157,6.629002,49.985197,40.415755
37,BAL,6/18/2021,1,669145,Bruce Zimmermann,SP,N,NaN,1,8,...,17.731343,1.394872,0.533333,0.861538,2.272727,11.283582,2.345865,6.511628,47.716013,40.498538
46,CLE,6/18/2021,1,650496,Jean Carlos Mejia,SP,N,NaN,4,4,...,0.000000,1.600000,0.266667,1.333333,0.285714,1.600000,0.500000,4.000000,5.350317,6.746736
55,PIT,6/18/2021,1,641771,Chad Kuhl,SP,N,NaN,3,5,...,7.500000,0.935252,0.683453,0.251799,1.116071,7.333333,1.190476,3.095238,30.028049,27.597833
65,OAK,6/18/2021,1,621076,James Kaprielian,SP,N,NaN,2,7,...,4.725000,2.245283,0.660377,1.584906,1.042553,6.300000,1.448276,6.034483,20.444587,31.223082
75,NYY,6/18/2021,1,592791,Jameson Taillon,SP,N,NaN,4,5,...,6.067416,1.271930,0.131579,1.140351,1.111111,4.550562,1.315789,2.777778,17.204750,18.737337
84,STL,6/18/2021,1,593372,Carlos Martinez,SP,N,NaN,3,5,...,7.714286,0.722892,0.481928,0.240964,1.357143,7.714286,1.403509,2.906977,29.479097,28.925530
93,ATL,6/18/2021,1,608331,Max Fried,SP,N,NaN,0,8,...,5.709251,1.894081,0.672897,1.221184,1.958042,10.255507,2.485437,6.511628,27.994271,30.623321


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [22]:
scores = pd.read_csv('../datasets/6_18_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210618,109z,545333,"Bauer, Trevor",1.0,9,1.0,32.15,10600.0,LAD,@ ari,NaN,3,0
1,20210618,106e,502171,"Cobb, Alex",1.0,0,1.0,29.75,8800.0,LAA,v det,NaN,11,3
2,20210618,151k,663978,"Paddack, Chris",1.0,9,1.0,29.05,8300.0,SDG,v cin,NaN,8,2
3,20210618,143n,608331,"Fried, Max",1.0,9,1.0,27.35,7700.0,ATL,v stl,NaN,9,1
4,20210618,127s,607074,"Rodon, Carlos",1.0,0,1.0,26.15,10800.0,CHW,@ hou,NaN,1,2


In [23]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [24]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [25]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [26]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [27]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [28]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [29]:
bat_final.to_csv('../merged_data/hitters_6_18.csv')
pitch_final.to_csv('../merged_data/pitchers_6_18.csv')

In [30]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,NYM,6/18/2021,1,542340,Jonathan Villar,1,N,NaN,R,Erick Fedde,...,1.0,1,5.0,0.0,4200.0,NYM,@ was,NaN,0,1
1,NYM,6/18/2021,1,596019,Francisco Lindor,2,N,NaN,R,Erick Fedde,...,1.0,2,6.0,0.0,4100.0,NYM,@ was,NaN,0,1
2,NYM,6/18/2021,1,642086,Dominic Smith,4,N,NaN,R,Erick Fedde,...,1.0,3,7.0,5.0,4400.0,NYM,@ was,NaN,0,1
3,NYM,6/18/2021,1,641856,Billy McKinney,6,N,NaN,R,Erick Fedde,...,1.0,5,7.0,0.0,3200.0,NYM,@ was,NaN,0,1
5,WAS,6/18/2021,1,607208,Trea Turner,1,N,NaN,L,Joey Lucchesi,...,1.0,2,6.0,3.0,4900.0,WAS,v nym,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,SEA,6/18/2021,1,572122,Kyle Seager,3,N,NaN,R,Michael Wacha,...,1.0,3,5.0,10.0,3600.0,SEA,v tam,NaN,5,1
181,SEA,6/18/2021,1,664034,Ty France,4,N,NaN,R,Michael Wacha,...,1.0,4,3.0,15.0,2700.0,SEA,v tam,NaN,5,1
183,SEA,6/18/2021,1,664238,Dylan Moore,6,N,NaN,R,Michael Wacha,...,1.0,6,4.0,2.0,3000.0,SEA,v tam,NaN,5,1
184,SEA,6/18/2021,1,641343,Jake Bauers,7,N,NaN,R,Michael Wacha,...,1.0,5,37.0,10.0,2300.0,SEA,v tam,NaN,5,1


In [31]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,NYM,6/18/2021,1,664192,Joey Lucchesi,SP,N,NaN,2,6,...,1.0,9,1.0,18.40,7000.0,NYM,@ was,NaN,0,1
1,WAS,6/18/2021,1,607200,Erick Fedde,SP,N,NaN,4,4,...,1.0,9,1.0,24.15,7500.0,WAS,v nym,NaN,1,0
2,TOR,6/18/2021,1,592662,Robbie Ray,SP,N,NaN,1,8,...,1.0,0,1.0,10.95,9300.0,TOR,@ bal,NaN,1,7
3,PIT,6/18/2021,1,641771,Chad Kuhl,SP,N,NaN,3,5,...,1.0,9,1.0,19.90,5900.0,PIT,v cle,NaN,11,10
4,OAK,6/18/2021,1,621076,James Kaprielian,SP,N,NaN,2,7,...,1.0,0,1.0,22.35,9100.0,OAK,@ nyy,NaN,5,3
5,NYY,6/18/2021,1,592791,Jameson Taillon,SP,N,NaN,4,5,...,1.0,0,1.0,12.90,7100.0,NYY,v oak,NaN,3,5
6,STL,6/18/2021,1,593372,Carlos Martinez,SP,N,NaN,3,5,...,1.0,9,1.0,-11.65,6700.0,STL,@ atl,NaN,1,9
7,ATL,6/18/2021,1,608331,Max Fried,SP,N,NaN,0,8,...,1.0,9,1.0,27.35,7700.0,ATL,v stl,NaN,9,1
8,MIN,6/18/2021,1,621244,Jose Berrios,SP,N,NaN,4,5,...,1.0,0,1.0,14.70,8500.0,MIN,@ tex,NaN,7,5(F10)
9,TEX,6/18/2021,1,592314,Mike Foltynewicz,SP,N,NaN,2,7,...,1.0,0,1.0,6.45,5700.0,TEX,v min,NaN,5,7(F10)
